In [1]:
import sys
sys.path.append("..")

import pickle
import numpy as np
import pandas as pd

# Helper classes for data processing, state representation
from reinforcetrader.data_pipeline import RawDataLoader, FeatureBuilder
from reinforcetrader.state import EpisodeStateLoader
from reinforcetrader.dqn_agent import DRLAgent
from reinforcetrader.backtest_engine import EDBacktester

# Helper method for displaying large dataframes
from IPython.display import HTML
pd.set_option("display.max_rows", None)
from IPython.display import HTML, display

def display_df(df, rows=10):
    # Puts the scrollbar next to the DataFrame
    display(HTML("<div style='height: 300px; overflow: auto; width: 98%'>" + df.head(rows).to_html() + "</div>"))

# Test DQN Agent Performance on OOS Data
To run the backtest, we must first set up a few things: replicate the WFV state loader, and load the model and its configuration.

## Get WFV State Loader

In [2]:
# Get DJI raw data using Yahoo Finance API
data_loader = RawDataLoader(start_date='1999-08-01', end_date='2025-09-08', index='DJI', verbose=False)

# Load the data from cache
ticker_data, benchmark_data = data_loader.get_hist_prices()

# Build features for state representation, reward computation, etc.
feature_builder = FeatureBuilder(ticker_data=ticker_data, benchmark_data=benchmark_data, f_prefix='DJI')
feature_builder.build_features()

# Store the computed features and the feature index ranges (used later)
features_data = feature_builder.get_features()
feature_indices = feature_builder.get_feature_indices()

Building ticker features: 100%|█████████████████████████████████████| 28/28 [00:05<00:00,  5.12it/s]

File already exists, skipping save: ../data/processed/DJI_tickers_features_2000-04-07_2025-09-04.csv


In [3]:
# Define the Walk-Forward Validation configuration
WFV_config = {
    'mode': 'expanding', # supports 'expanding' and 'moving'
    'train_start': '2000-04-07',
    'train_end': '2019-09-30',
    'test_start': '2019-10-01',
    'test_end': '2025-07-31',
    'train_window_size': 1200,
    'val_window_size': 630
}

In [4]:
# Initialize the episode state loader
state_loader = EpisodeStateLoader(features_data, feature_indices, WFV_config)

## Load the Trained Agent

In [5]:
# Create RL agent and testing configuration
# NOTE: Many of the params are NOT used in testing, but are used to define the agent. Thus, defining them is optional
agent_config = {
    'num_features': len(feature_indices['State']),
    'learning_rate': 1e-3,
    'dropout_p': 0.1,
    'memory_buffer_len': 200000,
    'state_matrix_window': 60,
    'epsilon_start': 1.0,
    'epsilon_min': 0.05,
    'epsilon_boost_factor': 0.3,
    'decay_updates': 50000,
    'discount_factor': 0.95,
    'batch_size': 256,
    'replay_start_size': 5000,
    'train_interval': 1,
}

test_config = {
    'plots_dir': '../plots/',
    'logs_dir': '../logs/',
    'outputs_dir': '../data/test/'
}

# Set the trained model path here
MODEL_PATH = '../model_checkpoints/DDQN_DSRC_AE1-44-44.keras'

# Load DRL Agent
trained_agent = DRLAgent(agent_config=agent_config, reward_type='DSR', model_path=MODEL_PATH)

I0000 00:00:1766960398.674185  100896 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 3584 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


Loading model from ../model_checkpoints/DDQN_DSRC_AE1-44-44.keras


## Test Agent Performance From 2020-2025

In [6]:
# Run event-driven backtest, i.e., run DRL agent and account for portfolio constraints during executions
backtester = EDBacktester(trained_agent, state_loader, index='DJI', cash=100_000, tc=0.25, max_pos=5)
portfolio_hist, trade_logs = backtester.run_backtest()

Starting Event-Driven Backtest. Initial Cash Balance: 100000.00; Max Positions: 5


Trading Days: 100%|█████████████████████████████████████████████████████| 1406/1406 [23:46<00:00,  1.01s/it]


In [7]:
# Compute the strategy and benchmark performance statistics
performance_stats = backtester.compute_performance_stats()
display(performance_stats)

/home/shiv/projects/ReinforceTrader/notebooks/../reinforcetrader/backtest_engine.py:253: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  index_returns = index_close.pct_change().fillna(0.0)


,Total Return,Ann. Return,Max Drawdown,Sharpe,Sortino
Strategy,,,,,
DRL Strategy,78.94%,10.99%,-34.27%,0.627,0.806
Index (Buy and Hold),56.52%,8.36%,-37.09%,0.492,0.591
EWP (Daily; No Trans Cost),123.11%,15.47%,-32.93%,0.816,0.990


In [8]:
# Plot the equity curves and benchmakrs
backtester.plot_curves()

## Agent Behavior Analysis

In [9]:
# Inspect the trade logs to identify a ticker and long position entry date
display_df(trade_logs)

,date,ticker,action,price,shares,cash_flow,net_profit
0,2019-12-30,DIS,BUY,140.462219,142.0,-19995.499220,NaN
1,2020-01-03,WMT,BUY,36.112190,555.0,-20092.371251,NaN
2,2020-01-10,HON,BUY,148.612076,134.0,-19963.803203,NaN
3,2020-01-13,CVX,BUY,89.267891,222.0,-19867.015466,NaN
4,2020-01-15,CAT,BUY,128.496658,155.0,-19966.774496,NaN
5,2020-02-04,CAT,SELL,118.677795,155.0,18349.070643,-1617.703853
6,2020-02-05,MMM,BUY,108.835503,169.0,-18439.182943,NaN
7,2020-02-07,WMT,SELL,35.671082,555.0,19747.956631,-344.414620
8,2020-02-10,AMGN,BUY,189.065842,100.0,-18953.850628,NaN
9,2020-02-11,HON,SELL,149.002960,134.0,19916.480676,-47.322528


In [10]:
# Let us pick MCD stock for analysis
SCENARIO_TICKER = 'MCD'
MCD_trade_logs = trade_logs[trade_logs['ticker'] == 'MCD']

# Inspect MCD buy/sell behaviour
display_df(MCD_trade_logs)

,date,ticker,action,price,shares,cash_flow,net_profit
102,2021-01-12,MCD,BUY,188.768616,131.0,-24790.510381,NaN
131,2021-05-17,MCD,SELL,207.953674,131.0,27173.826507,2383.316126
168,2022-01-27,MCD,BUY,226.981918,116.0,-26395.727283,NaN
171,2022-02-04,MCD,SELL,237.311722,116.0,27459.339330,1063.612047
213,2022-08-26,MCD,BUY,237.084824,115.0,-27332.916602,NaN
217,2022-09-08,MCD,SELL,241.286377,115.0,27678.563516,345.646914
294,2023-09-06,MCD,BUY,261.228241,119.0,-31163.876077,NaN
298,2023-09-18,MCD,SELL,263.390594,119.0,31265.122042,101.245965
381,2024-10-25,MCD,BUY,284.230774,123.0,-35047.786156,NaN
391,2025-01-06,MCD,SELL,285.520355,123.0,35031.206183,-16.579972


In [12]:
# Set buy/sell date for agent behaviour simulation
BUY_DATE = pd.to_datetime('2021-01-12')

# Run the trade scenario and visualize
# Store the states and reward for all time steps from buy to sell
trade_states, trade_rewards = backtester.get_trade_scenario(BUY_DATE, SCENARIO_TICKER)

In [13]:
# Store the MCD states and reward computes locally
def np_save(idf1, idf2, obj_type: str, np_obj: np.ndarray) -> None:
    f_path = f'../data/test/states/{idf1}_{idf2}_{obj_type}.npy'
    np.save(f_path, np_obj)

np_save(SCENARIO_TICKER, BUY_DATE.strftime("%Y_%m_%d"), 'states', trade_states)
np_save(SCENARIO_TICKER, BUY_DATE.strftime("%Y_%m_%d"), 'rewards', trade_rewards)

# For explanability use cases, we store random samples from the test window
num_rand_samples = 800
rand_states, rand_rewards= backtester.get_random_test_samples(num_states=num_rand_samples)
np_save('rand', num_rand_samples, 'states', rand_states)
np_save('rand', num_rand_samples, 'rewards', rand_rewards)